In [24]:
import os
import numpy as np
import pandas as pd


def create_csv(input, output):
  df = pd.read_csv(input)

  obj = {"img_width": df['image_width'].iloc[0],
         "img_height": df['image_height'].iloc[0],
         "patch_w": df['patch_width'].iloc[0],
         "patch_h": df['patch_height'].iloc[0],
         "png_w": df['image_width'].iloc[0] / df['patch_width'].iloc[0],
         "png_h": df['image_height'].iloc[0] / df['patch_height'].iloc[0]}

  cols = list(df.columns)
  modified = df[cols[5:7] + cols[11:12]]
  modified.sort_values(['patch_x', 'patch_y'], ascending=[1, 1])
  modified['i'] = modified['patch_x'] / df['patch_width']
  modified['j'] = modified['patch_y'] / df['patch_height']
  modified['n'] = modified['nuclei_ratio'] * 255

  modified.i = np.ceil(modified.i).astype(int)
  modified.j = np.ceil(modified.j).astype(int)
  modified.n = np.ceil(modified.n).astype(int)

  
        
  modified.drop("nuclei_ratio", axis=1, inplace=True)
  modified = modified.rename(index=str, columns={"n": "nuclei_ratio"})

  cols = list(modified.columns)
  modified = modified[cols[2:]]
  # print(modified)
  modified.to_csv(output, index=False)



cwd = os.getcwd()
for filename in os.listdir(cwd):
  if filename.endswith(".csv"):
    create_csv(os.path.join(cwd, filename), filename + '.1')


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers